In [ ]:
import time
%run simple_interval_propagation.ipynb

In [ ]:
# description: find a counter example in the TODO list, start to search with larger intervals
# input parameter nn model: a keras model
# input parameter all_regions: a list that contains all the regions that are still in the waiting list
# input parameter y_true: the real output value for the original input interval
# return counter_example_list: a list that contains all the counter examples(counter points)
def if_any_counter_example_model_2_large_first(nn_model, all_regions, y_true):
    counter_example = []
    for region in all_regions:
        region_center_point = find_center_point(region)
        predict_prob = nn_model.predict(region_center_point)
        if predict_prob > 0.5:
            predict_res = 1
        else:
            predict_res = 0
        if predict_res != y_true:  
            counter_example.append(region_center_point)
            break
    return counter_example

In [ ]:
# description: find a counter example in the TODO list, start to search with smaller intervals
# input parameter nn model: a keras model
# input parameter all_regions: a list that contains all the regions that are still in the waiting list
# input parameter y_true: the real output value for the original input interval
# return counter_example_list: a list that contains all the counter examples(counter points)
def if_any_counter_example_model_2_small_first(nn_model, all_regions, y_true):
    counter_example = []
    for region in reversed(all_regions):
        region_center_point = find_center_point(region)
        predict_prob = nn_model.predict(region_center_point)
        if predict_prob >= 0.5:
            predict_res = 1
        else:
            predict_res = 0
        if predict_res != y_true: 
            counter_example.append(region_center_point)
            break
    return counter_example

In [ ]:
# description: check specification of model 2 for the output interval. because we control the input interval is under the x1=x2, the output probability should all smaller the 0.5
# input parameter output_min, output_max: minimum and maximum value vectors to define the output interval, after the simple interval propagation
# return true if 
def check_specification_model_2(output_max):
    if output_max < 0.5:
        return True
    else:
        return False

In [ ]:
# description: find all the counter examples in the TODO list 
# input parameter nn model: a keras model
# input parameter all_regions: a list that contains all the regions that are still in the waiting list
# return counter_example_list: a list that contains all the counter examples(counter points)
def find_counter_examples_in_all_regions_in_list_model_2(nn_model, all_regions, y_true):
    counter_example_list = []
    for region in all_regions:
        region_center_point = find_center_point(region)
        predict_prob = nn_model.predict(region_center_point)
        if predict_prob > 0.5:
            predict_res = 1
        else:
            predict_res = 0
        if predict_res != y_true:
            counter_example_list.append(region_center_point)
    return counter_example_list

In [ ]:
# description: generate the point_list is in type of python list contains 100 point within x1[0., 100.], x2[0., 100.]
# the input parameters num: the number of input wants to generate
# return point_list: a list of number of points
def get_point_list_model_2(num, interval):
    np.random.seed(1)
    point_list = []
    for i in range(num):
        x1 = np.random.uniform(interval[0][0], interval[1][0])
        x2 = np.random.uniform(interval[0][1], interval[1][1])
        point_list.append(np.array([[x1, x2]]))
    return point_list

In [ ]:
# description: check the property of model 2 is satisfy or not
# the input parameters y_true: the real y value from the predefined property
# the input parameters output_min, output_max: the output interval that computed by the plain interval propagation function
# return true or false: the property is satify by the output interval or not

def check_property_model_2(y_true, output_min, output_max):
    if y_true:
        if output_min >= 0.5:
            return True
        else:
            return False
    else:
        if output_max <= 0.5:
            return True
        else:
            return False

In [ ]:
# description: certify model 2 with a single interval as input. larger sub interval in the TODO list priority
# the input parameters point_list: a list contains all the points
# the input parameters max_perturbation: the maximum perturbation of l infinity norm, to generate the interval
# return output_l_infinity_interval_list: a list of all intervals that are performed l infinity norm
def certify_model_2_with_single_input_interval_larger_interval_pri(orig_input_interval, y, total_seconds):
    
    start_time = time.time()
    
    list_fifo = []
    list_fifo.append(orig_input_interval)
    counter_example = []
    
    while True:
        elapsed_time = time.time() - start_time
        
        if not list_fifo:
            result = {
                "sat": 1,
                "counter_example": None,
            }
            return result
        if elapsed_time > total_seconds:
            counter_example = if_any_counter_example_model_2_large_first(model_2, list_fifo, y)
            if counter_example:
                result = {
                    "sat": 0,
                    "counter_example": counter_example,
                }
                return result
            else:
                result = {
                    "sat": 0,
                    "counter_example": None,
                }
                
        input_region = list_fifo.pop(0)
        input_min = input_region[0]
        input_max = input_region[1]
        outputMin, outputMax = simple_interval_propagation_model(model_2, input_min, input_max)
        
        if not check_property_model_2(y, outputMin, outputMax):
            interval1, interval2 = split_2(input_min, input_max)
            list_fifo.append(interval1)
            list_fifo.append(interval2)

In [ ]:
# description: certify model 2 with a single interval as input. larger sub interval in the TODO list priority
# the input parameters point_list: a list contains all the points
# the input parameters max_perturbation: the maximum perturbation of l infinity norm, to generate the interval
# return output_l_infinity_interval_list: a list of all intervals that are performed l infinity norm
def certify_model_2_with_single_input_interval_larger_interval_pri_search_smaller_pri(orig_input_interval, y, total_seconds):
    
    start_time = time.time()
    
    list_fifo = []
    list_fifo.append(orig_input_interval)
    counter_example = []
    
    while True:
        elapsed_time = time.time() - start_time
        
        if not list_fifo:
            result = {
                "sat": 1,
                "counter_example": None,
            }
            return result
        if elapsed_time > total_seconds:
            counter_example = if_any_counter_example_model_2_small_first(model_2, list_fifo, y)
            if counter_example:
                result = {
                    "sat": 0,
                    "counter_example": counter_example,
                }
                return result
            else:
                result = {
                    "sat": 0,
                    "counter_example": None,
                }
                
        input_region = list_fifo.pop(0)
        input_min = input_region[0]
        input_max = input_region[1]
        outputMin, outputMax = simple_interval_propagation_model(model_2, input_min, input_max)
        
        if not check_property_model_2(y, outputMin, outputMax):
            interval1, interval2 = split_2(input_min, input_max)
            list_fifo.append(interval1)
            list_fifo.append(interval2)

In [ ]:
# description: certify model 2 with a single interval as input. Smaller sub interval in the TODO list priority
# the input parameters point_list: a list contains all the points
# the input parameters max_perturbation: the maximum perturbation of l infinity norm, to generate the interval
# return output_l_infinity_interval_list: a list of all intervals that are performed l infinity norm
def certify_model_2_with_single_input_interval_smaller_interval_pri(orig_input_interval, y, total_seconds):
    
    start_time = time.time()
    
    list_lifo = []
    list_lifo.append(orig_input_interval)
    counter_example = []
    
    while True:
        elapsed_time = time.time() - start_time
        
        if not list_lifo:
            result = {
                "sat": 1,
                "counter_example": None,
            }
            return result
        if elapsed_time > total_seconds:
            counter_example = if_any_counter_example_model_2_large_first(model_2, list_lifo, y)
            if counter_example:
                result = {
                    "sat": 0,
                    "counter_example": counter_example,
                }
                return result
            else:
                result = {
                    "sat": 0,
                    "counter_example": None,
                }
                
        input_region = list_lifo.pop(len(list_lifo)-1)
        input_min = input_region[0]
        input_max = input_region[1]
        outputMin, outputMax = simple_interval_propagation_model(model_2, input_min, input_max)
        
        if not check_property_model_2(y, outputMin, outputMax):
            interval1, interval2 = split_2(input_min, input_max)
            list_lifo.append(interval1)
            list_lifo.append(interval2)

In [ ]:
# description: certify model 2 with a single interval as input. Smaller sub interval in the TODO list priority
# the input parameters point_list: a list contains all the points
# the input parameters max_perturbation: the maximum perturbation of l infinity norm, to generate the interval
# return output_l_infinity_interval_list: a list of all intervals that are performed l infinity norm
def certify_model_2_with_single_input_interval_smaller_interval_pri_search_smaller_pri(orig_input_interval, y, total_seconds):
    
    start_time = time.time()
    
    list_lifo = []
    list_lifo.append(orig_input_interval)
    counter_example = []
    
    while True:
        elapsed_time = time.time() - start_time
        
        if not list_lifo:
            result = {
                "sat": 1,
                "counter_example": None,
            }
            return result
        if elapsed_time > total_seconds:
            counter_example = if_any_counter_example_model_2_small_first(model_2, list_lifo, y)
            if counter_example:
                result = {
                    "sat": 0,
                    "counter_example": counter_example,
                }
                return result
            else:
                result = {
                    "sat": 0,
                    "counter_example": None,
                }
                
        input_region = list_lifo.pop(len(list_lifo)-1)
        input_min = input_region[0]
        input_max = input_region[1]
        outputMin, outputMax = simple_interval_propagation_model(model_2, input_min, input_max)
        
        if not check_property_model_2(y, outputMin, outputMax):
            interval1, interval2 = split_2(input_min, input_max)
            list_lifo.append(interval1)
            list_lifo.append(interval2)

In [ ]:
# description: certify model 2 with a certain input interval and 100 random intervals within this certain input interval
# the input parameters original_intervals: a list contains all the intervals
# the input parameters y_true: the output property of all the intervals. all the intervals should has one same property
# the input parameters single_interval_certification_time: the total run time for the certification of each interval
# return result: 
def certify_model_2_larger_interval_pri_with_100_intervals(original_interval_list, y_true, single_interval_certification_time):
    counter_interval_list = []
    counter_example_list = [] 
    interval_nums = len(original_interval_list)
    
    for i in range(interval_nums):
        single_result = certify_model_2_with_single_input_interval_larger_interval_pri(original_interval_list[i], y_true, single_interval_certification_time)
        if single_result["sat"] == 0:
            counter_interval_list.append(original_interval_list[i])
            counter_example_list.append(single_result["counter_example"])
    result = {
        "sat_num": len(counter_interval_list),
        "counter_interval_list": counter_interval_list,
        "counter_examples": counter_example_list
    }
    
    return result

In [ ]:
#********************************larger interval priority, counter example search smaller priority*************************************************************
def certify_model_2_larger_interval_pri_search_smaller_pri_with_1000_intervals(original_interval_list, y_true, single_interval_certification_time):
    counter_interval_list = []
    counter_example_list = [] 
    interval_nums = len(original_interval_list)
    
    for i in range(interval_nums):
        single_result = certify_model_2_with_single_input_interval_larger_interval_pri_search_smaller_pri(original_interval_list[i], y_true, single_interval_certification_time)
        if single_result["sat"] == 0:
            counter_interval_list.append(original_interval_list[i])
            counter_example_list.append(single_result["counter_example"])
    result = {
        "sat_num": len(counter_interval_list),
        "counter_interval_list": counter_interval_list,
        "counter_examples": counter_example_list
    }
    
    return result

In [ ]:
def certify_model_2_smaller_interval_pri_with_1000_intervals(original_interval_list, y_true, single_interval_certification_time):
    counter_interval_list = []
    counter_example_list = [] 
    interval_nums = len(original_interval_list)
    
    for i in range(interval_nums):
        single_result = certify_model_2_with_single_input_interval_smaller_interval_pri(original_interval_list[i], y_true, single_interval_certification_time)
        if single_result["sat"] == 0:
            counter_interval_list.append(original_interval_list[i])
            counter_example_list.append(single_result["counter_example"])
    result = {
        "sat_num": len(counter_interval_list),
        "counter_interval_list": counter_interval_list,
        "counter_examples": counter_example_list
    }
    
    return result

In [ ]:
def certify_model_2_smaller_interval_pri_search_smaller_pri_with_1000_intervals(original_interval_list, y_true, single_interval_certification_time):
    counter_interval_list = []
    counter_example_list = [] 
    interval_nums = len(original_interval_list)
    
    for i in range(interval_nums):
        single_result = certify_model_2_with_single_input_interval_smaller_interval_pri_search_smaller_pri(original_interval_list[i], y_true, single_interval_certification_time)
        if single_result["sat"] == 0:
            counter_interval_list.append(original_interval_list[i])
            counter_example_list.append(single_result["counter_example"])
    result = {
        "sat_num": len(counter_interval_list),
        "counter_interval_list": counter_interval_list,
        "counter_examples": counter_example_list
    }
    
    return result

In [ ]:
def is_counter_example_model_2(point, y_true):
    predict_val = model_2.predict(point)
    if predict_val < 0.5:
        predict_val = 0
    else:
        predict_val = 1
    if y_true == predict_val:
        counter_example = None
    else:
        counter_example = point
    return counter_example

In [ ]:
def certify_model_2_with_single_input_interval_plain_interval_propagation(original_interval, y_true):
    
    
    output_min, output_max = simple_interval_propagation_model(model_2, original_interval[0], original_interval[1])
    
    if y_true == 1:
        if output_min >= 0.5:
            sat = 1
        else:
            sat = 0

    if y_true == 0:
        if output_max <= 0.5:
            sat = 1
        else:
            sat = 0
        
    result = {
        "sat": sat,
        "counter_example": None
    }
    return result

In [ ]:
def certify_model_2_with_single_input_interval_plain_interval_propagation_with_counter_example(original_interval, y_true):
    
    
    output_min, output_max = simple_interval_propagation_model(model_2, original_interval[0], original_interval[1])
    
    if y_true == 1:
        if output_min >= 0.5:
            sat = 1
        else:
            sat = 0

    if y_true == 0:
        if output_max <= 0.5:
            sat = 1
        else:
            sat = 0
            
    center_point = find_center_point(original_interval)
    counter_example = is_counter_example_model_2(center_point, y_true)
        
    result = {
        "sat": sat,
        "counter_example": counter_example
    }
    return result

In [ ]:
def certify_model_2_with_1000_intervals_plain_interval_propagation(original_interval_list, y_true):
    counter_interval_list = []
    counter_example_list = [] 
    interval_nums = len(original_interval_list)
    
    for i in range(interval_nums):
        single_result = certify_model_2_with_single_input_interval_plain_interval_propagation(original_interval_list[i], y_true)
        if single_result["sat"] == 0:
            counter_interval_list.append(original_interval_list[i])
            # counter_example_list.append(single_result["counter_example"])
    result = {
        "sat_num": len(counter_interval_list),
        "counter_interval_list": counter_interval_list,
        "counter_examples": counter_example_list
    }
    
    return result

In [ ]:
def certify_model_2_with_1000_intervals_plain_interval_propagation_with_counter_example(original_interval_list, y_true):
    counter_interval_list = []
    counter_example_list = [] 
    interval_nums = len(original_interval_list)
    
    for i in range(interval_nums):
        single_result = certify_model_2_with_single_input_interval_plain_interval_propagation_with_counter_example(original_interval_list[i], y_true)
        if single_result["sat"] == 0:
            counter_interval_list.append(original_interval_list[i])
            counter_example_list.append(single_result["counter_example"])
    result = {
        "sat_num": len(counter_interval_list),
        "counter_interval_list": counter_interval_list,
        "counter_examples": counter_example_list
    }
    
    return result